In [35]:
#importing packages
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
    from collections import Counter, defaultdict
    import math
    import json
    import numpy as np
    import os
    import string
    import pandas as pd
    import re
    import pydotplus
    from sklearn import tree, preprocessing
    from sklearn.tree import _tree
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.decomposition import TruncatedSVD
    from sklearn.preprocessing import Normalizer
    from sklearn.pipeline import make_pipeline
    from sklearn.neighbors import NearestNeighbors
    from sklearn_pandas import DataFrameMapper, cross_val_score
    from nltk.corpus import stopwords
    from nltk.stem.snowball import SnowballStemmer
    from scipy.sparse import csr_matrix
    from IPython.display import Image  
    


In [3]:
# Read the input
file = "reformatted1.csv"
df = pd.read_csv(file, na_values = "null")
# the consumer dataset is now a Pandas DataFrame
# Only interested in data with consumer complaints
d = df[df['Consumer complaint narrative'].notnull()]

/Users/jponder/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (13,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
d = d[~d['Company response to consumer'].isin(['Closed', 'In progress', 'Untimely response'])]
df2 = df[~df['Company response to consumer'].isin(['Closed', 'In progress', 'Untimely response'])]
print(df2[['Company response to consumer']].groupby(['Company response to consumer']).size())

print(d.columns.values)
grouped = d[['Company response to consumer']].groupby(['Company response to consumer'])
print(d[['Company response to consumer']].groupby(['Company response to consumer']).size())
print(d[['Product', 'Sub-product', 'Issue', 'Sub-issue']].groupby(['Product', 'Sub-product', 'Issue', 'Sub-issue']).size())

Company response to consumer
Closed with explanation            553421
Closed with monetary relief         49610
Closed with non-monetary relief     92111
Closed with relief                   5298
Closed without relief               17863
dtype: int64
['sid' 'id' 'position' 'created_at' 'created_meta' 'updated_at'
 'updated_meta' 'meta' 'Date received' 'Product' 'Sub-product' 'Issue'
 'Sub-issue' 'Consumer complaint narrative' 'Company public response'
 'Company' 'State' 'ZIP code' 'Tags' 'Consumer consent provided?'
 'Submitted via' 'Date sent to company' 'Company response to consumer'
 'Timely response?' 'Consumer disputed?' 'Complaint ID']
Company response to consumer
Closed with explanation            112233
Closed with monetary relief         10461
Closed with non-monetary relief     17497
dtype: int64
Product          Sub-product                     Issue                                     Sub-issue                               
Debt collection  Auto                            

In [ ]:
def predict(entry, cls, values):
    prediction = cls.predict_proba(entry)
    leaf = cls.apply(entry)
    classifier = np.argmax(prediction)
    samples = cls.tree_.n_node_samples
    confident = False
    prediction_class = values[classifier]
    if prediction[0][classifier] == 1 and samples[leaf] > 50:
        confident = True 
    return (prediction_class, confident)

def create_new_features(data):
    data['combined_product'] = data.apply(lambda x: "%s-%s" % (x['Product'], x['Sub-product']), axis = 1)
    data['combined_issue'] = data.apply(lambda x: "%s-%s" % (x['Issue'], x['Sub-issue']), axis = 1)
    
def shape_data(data):
    decision_tree_data = data.ix[:, ['combined_product', 'combined_issue', 'State', 'Company']]
    return pd.get_dummies(decision_tree_data)

def split_data(data):
    data['Date received'] = pd.to_datetime(data['Date received'])
    data = data.sort_values('Date received')
    cutoff = int(data.shape[0] * 0.8)
    return(data, cutoff)
            
def run_prediction(data):
    create_new_features(data)
    data, cutoff = split_data(data)
    shaped_data = shape_data(data)
    train = data.iloc[:cutoff]
    test = data.iloc[cutoff +1:]
    train_shaped = shaped_data.iloc[:cutoff]
    test_shaped = shaped_data.iloc[cutoff +1:]
    print(test[['Company response to consumer']].groupby(['Company response to consumer']).size())
    clf = tree.DecisionTreeClassifier()
    target = train['Company response to consumer'].values
    clf = clf.fit( train_shaped.values, y = target )
    train['Prediction'] = train_shaped.apply(lambda x: predict(x, clf, clf.classes_), axis = 1)
    train['Confident'] = train.apply(lambda x: x.Prediction[1], axis = 1)
    train['Prediction'] = train.apply(lambda x: x.Prediction[0], axis = 1)
    test['Prediction'] = test_shaped.apply(lambda x: predict(x, clf, clf.classes_), axis = 1)
    test['Confident'] = test.apply(lambda x: x.Prediction[1], axis = 1)
    test['Prediction'] = test.apply(lambda x: x.Prediction[0], axis = 1)
    return (train, test)
    
train, test = run_prediction(df2)

In [6]:
train.tail()

,sid,id,position,created_at,created_meta,updated_at,updated_meta,meta,Date received,Product,...,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,combined_product,combined_issue,Prediction,Confident
626638,665894,BEEDDC38-243A-4808-85E2-0DC771586EBA,665894,1472495798,912605,1472495798,912605,NaN,2016-06-21 21:56:21,Bank account or service,...,Web,2016-06-21T21:56:21,Closed with monetary relief,Yes,No,1978860,Bank account or service-Checking account,Problems caused by my funds being low-nan,Closed with explanation,False
688993,835499,65E2063D-75CC-424C-8835-2ECE92EF1156,835499,1481963458,912605,1481963458,912605,NaN,2016-06-21 21:56:39,Debt collection,...,Postal mail,2016-08-18T17:23:24,Closed with non-monetary relief,Yes,No,1978989,Debt collection-Other (i.e. phone; health club...,Disclosure verification of debt-Not given enou...,Closed with explanation,False
655996,746298,916BFC93-C91D-4CEE-9479-83AEE700F08A,746298,1477099606,912605,1477099606,912605,NaN,2016-06-21 21:56:55,Student loan,...,Web,2016-08-09T11:11:12,Closed with non-monetary relief,Yes,No,1978450,Student loan-Federal student loan servicing,Dealing with my lender or servicer-Keep gettin...,Closed with non-monetary relief,False
627129,666404,1C58EDDA-2D59-4180-93B7-13906BB75746,666404,1472495802,912605,1472495802,912605,NaN,2016-06-21 22:00:35,Debt collection,...,Web,2016-06-21T22:00:35,Closed with non-monetary relief,Yes,No,1978123,Debt collection-Auto,Communication tactics-Threatened to take legal...,Closed with non-monetary relief,False
631379,676989,6BA6C7BD-9BEA-4AEB-940B-4343CABDFBF4,676989,1473233922,912605,1473233922,912605,NaN,2016-06-21 22:02:01,Credit card,...,Web,2016-06-30T15:51:23,Closed with non-monetary relief,Yes,No,1978305,Credit card-nan,Billing statement-nan,Closed with explanation,False


In [7]:
test.tail()

,sid,id,position,created_at,created_meta,updated_at,updated_meta,meta,Date received,Product,...,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,combined_product,combined_issue,Prediction,Confident
623669,1011760,A8DD50CE-8FE7-478F-BB86-F32DA45F03E8,1011760,1490626002,912605,1490626002,912605,NaN,2017-03-24 18:13:28,Consumer Loan,...,Web,2017-03-24T18:29:37,Closed with explanation,Yes,NaN,2403063,Consumer Loan-Installment loan,Managing the loan or lease-nan,Closed with explanation,True
625633,1012929,13709271-37B1-49A5-9836-8C54F022009D,1012929,1490626005,912605,1490626005,912605,NaN,2017-03-24 21:18:11,Debt collection,...,Web,2017-03-24T21:18:12,Closed with explanation,Yes,Yes,2402762,Debt collection-Medical,Disclosure verification of debt-Right to dispu...,Closed with non-monetary relief,False
625146,1012722,1088D7C8-173C-4DE7-8972-E24BD2E7518E,1012722,1490626004,912605,1490626004,912605,NaN,2017-03-24 23:06:28,Debt collection,...,Web,2017-03-24T23:06:28,Closed with explanation,Yes,NaN,2403874,Debt collection-Other (i.e. phone; health club...,Cont'd attempts collect debt not owed-Debt is ...,Closed with explanation,True
624679,1012453,3C4EBDBA-0803-460A-844F-28938061EC30,1012453,1490626004,912605,1490626004,912605,NaN,2017-03-25 09:46:59,Debt collection,...,Web,2017-03-25T09:46:59,Closed with explanation,Yes,NaN,2403955,Debt collection-Credit card,Cont'd attempts collect debt not owed-Debt is ...,Closed with explanation,True
622764,1011021,9C43ED01-AEA8-4644-A9AD-C2AE3FE72A9B,1011021,1490626000,912605,1490626000,912605,NaN,2017-03-25 13:06:02,Debt collection,...,Web,2017-03-25T13:06:02,Closed with explanation,Yes,NaN,2404142,Debt collection-Other (i.e. phone; health club...,Cont'd attempts collect debt not owed-Debt was...,Closed with explanation,True


In [8]:
print(test.shape)
confident_predicted = test[test.Confident]
unconfident_predicted = test[test['Consumer complaint narrative'].isnull()]
unconfident_predicted = unconfident_predicted[~unconfident_predicted.Confident]
unpredicted = test[test['Consumer complaint narrative'].notnull()]
unpredicted = unpredicted[~unpredicted.Confident]
new_training = train[train['Consumer complaint narrative'].notnull()]
new_training = new_training[~new_training.Confident]
print(unpredicted[['Company response to consumer']].groupby(['Company response to consumer']).size())
print(confident_predicted[['Company response to consumer']].groupby(['Company response to consumer']).size())


bad_predictions = test[test['Prediction'] != test['Company response to consumer']]
print(bad_predictions.shape)
print(new_training.shape)


(143660, 30)
Company response to consumer
Closed with explanation            31231
Closed with monetary relief         3873
Closed with non-monetary relief     5773
dtype: int64
Company response to consumer
Closed with explanation            30968
Closed with monetary relief          467
Closed with non-monetary relief     1152
dtype: int64
(27746, 30)
(65358, 30)


In [9]:
pd.crosstab(confident_predicted['Company response to consumer'], confident_predicted['Prediction'])

Prediction,Closed with explanation,Closed with non-monetary relief
Company response to consumer,,
Closed with explanation,30950,18
Closed with monetary relief,467,0
Closed with non-monetary relief,719,433


In [10]:
pd.crosstab(test['Company response to consumer'], test['Prediction'])

Prediction,Closed with explanation,Closed with monetary relief,Closed with non-monetary relief,Closed with relief,Closed without relief
Company response to consumer,,,,,
Closed with explanation,110056,3135,4408,72,190
Closed with monetary relief,7005,1696,317,14,31
Closed with non-monetary relief,12113,435,4162,4,22


In [11]:
pd.crosstab(unconfident_predicted['Company response to consumer'], unconfident_predicted['Prediction'])

Prediction,Closed with explanation,Closed with monetary relief,Closed with non-monetary relief,Closed with relief,Closed without relief
Company response to consumer,,,,,
Closed with explanation,50848,1859,2800,43,112
Closed with monetary relief,3674,877,147,8,17
Closed with non-monetary relief,7188,251,2356,1,15


In [12]:
print(new_training.shape)
new_training.tail()

(65358, 30)


,sid,id,position,created_at,created_meta,updated_at,updated_meta,meta,Date received,Product,...,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,combined_product,combined_issue,Prediction,Confident
621675,658678,8427C612-05E7-4D74-9065-D856ECF903BE,658678,1472051514,912605,1472051514,912605,NaN,2016-06-21 21:49:45,Debt collection,...,Web,2016-06-22T14:28:17,Closed with non-monetary relief,Yes,No,1977764,Debt collection-I do not know,Cont'd attempts collect debt not owed-Debt is ...,Closed with non-monetary relief,False
620861,657524,7B81DA3E-30CE-4C91-8EA7-795D1A2A9BE3,657524,1472051509,912605,1472051509,912605,NaN,2016-06-21 21:50:10,Credit reporting,...,Web,2016-06-21T21:50:10,Closed with explanation,Yes,No,"1977766""",Credit reporting-nan,Incorrect information on credit report-Account...,Closed with explanation,False
626638,665894,BEEDDC38-243A-4808-85E2-0DC771586EBA,665894,1472495798,912605,1472495798,912605,NaN,2016-06-21 21:56:21,Bank account or service,...,Web,2016-06-21T21:56:21,Closed with monetary relief,Yes,No,1978860,Bank account or service-Checking account,Problems caused by my funds being low-nan,Closed with explanation,False
627129,666404,1C58EDDA-2D59-4180-93B7-13906BB75746,666404,1472495802,912605,1472495802,912605,NaN,2016-06-21 22:00:35,Debt collection,...,Web,2016-06-21T22:00:35,Closed with non-monetary relief,Yes,No,1978123,Debt collection-Auto,Communication tactics-Threatened to take legal...,Closed with non-monetary relief,False
631379,676989,6BA6C7BD-9BEA-4AEB-940B-4343CABDFBF4,676989,1473233922,912605,1473233922,912605,NaN,2016-06-21 22:02:01,Credit card,...,Web,2016-06-30T15:51:23,Closed with non-monetary relief,Yes,No,1978305,Credit card-nan,Billing statement-nan,Closed with explanation,False


In [13]:
unpredicted.tail()

,sid,id,position,created_at,created_meta,updated_at,updated_meta,meta,Date received,Product,...,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,combined_product,combined_issue,Prediction,Confident
623378,1011507,BB3F28BA-D61E-44E5-A37A-4C01A851FAA7,1011507,1490626002,912605,1490626002,912605,NaN,2017-03-19 03:31:19,Credit reporting,...,Web,2017-03-19T03:31:20,Closed with explanation,Yes,NaN,2393540,Credit reporting-nan,Unable to get credit report/credit score-Probl...,Closed with non-monetary relief,False
622857,1011079,9BE91B24-F1C4-4A63-BE54-27F020659DBC,1011079,1490626000,912605,1490626000,912605,NaN,2017-03-19 17:43:38,Credit card,...,Web,2017-03-19T17:43:38,Closed with explanation,Yes,NaN,"2393981""",Credit card-nan,Convenience checks-nan,Closed with monetary relief,False
622979,1011181,034C182E-36EC-458E-AA0D-7A4398CF152F,1011181,1490626000,912605,1490626000,912605,NaN,2017-03-19 19:08:13,Debt collection,...,Web,2017-03-19T19:08:13,Closed with non-monetary relief,Yes,NaN,2393976,Debt collection-Other (i.e. phone; health club...,Disclosure verification of debt-Not given enou...,Closed with non-monetary relief,False
627449,1013441,F3EBEAD9-049C-4B45-B4D6-B41D925D902C,1013441,1490626005,912605,1490626005,912605,NaN,2017-03-20 11:12:52,Credit reporting,...,Web,2017-03-20T11:12:53,Closed with explanation,Yes,NaN,2395821,Credit reporting-nan,Credit monitoring or identity protection-Probl...,Closed with explanation,False
625730,1012954,72C8CBDA-F83B-45D7-BCB9-EECA231A4FD7,1012954,1490626005,912605,1490626005,912605,NaN,2017-03-20 15:31:42,Credit reporting,...,Web,2017-03-20T15:31:43,Closed with explanation,Yes,NaN,2395641,Credit reporting-nan,Incorrect information on credit report-Reinser...,Closed with explanation,False


In [63]:
def lsa(data, pipeline=None, dimensions=200, short=False):
    stemmer = SnowballStemmer("english")
    table = str.maketrans({key: None for key in string.punctuation})
    stemmed = data['Consumer complaint narrative']
    stemmed = stemmed.map(lambda x: ' '.join([stemmer.stem(y.lower().strip('xx')) for y in x.translate(table).split(' ')]))
    if not pipeline:
        vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2), sublinear_tf=True, use_idf=True)
        svd = TruncatedSVD(dimensions)
        normalizer = Normalizer(copy=False)
        pipeline = make_pipeline(vectorizer, svd, normalizer).fit(stemmed)
    vectorized = pd.DataFrame(pipeline.transform(stemmed))
    data = data.reset_index()
    if not short:
        data = pd.concat([data, vectorized], axis=1)
    else:
        data = pd.concat([data['Company response to consumer'], vectorized], axis=1)
    return (pipeline, data)


In [15]:
pipeline, vectorized_train = lsa(new_training)

vectorized_train.tail()

,index,sid,id,position,created_at,created_meta,updated_at,updated_meta,meta,Date received,...,190,191,192,193,194,195,196,197,198,199
65353,621675,658678,8427C612-05E7-4D74-9065-D856ECF903BE,658678,1472051514,912605,1472051514,912605,NaN,2016-06-21 21:49:45,...,-0.049604,0.012788,0.011784,0.014123,0.049322,0.062420,0.037842,0.012093,0.036449,0.012021
65354,620861,657524,7B81DA3E-30CE-4C91-8EA7-795D1A2A9BE3,657524,1472051509,912605,1472051509,912605,NaN,2016-06-21 21:50:10,...,-0.015567,-0.011342,0.016717,0.022115,-0.054008,0.044485,-0.003411,-0.009678,-0.035649,-0.032809
65355,626638,665894,BEEDDC38-243A-4808-85E2-0DC771586EBA,665894,1472495798,912605,1472495798,912605,NaN,2016-06-21 21:56:21,...,0.100985,0.019490,0.027157,0.065313,0.043357,-0.055749,0.054356,0.031014,0.040227,0.111181
65356,627129,666404,1C58EDDA-2D59-4180-93B7-13906BB75746,666404,1472495802,912605,1472495802,912605,NaN,2016-06-21 22:00:35,...,-0.035119,-0.073786,-0.037980,0.056364,0.027337,0.057586,0.009682,-0.014140,-0.021107,-0.052996
65357,631379,676989,6BA6C7BD-9BEA-4AEB-940B-4343CABDFBF4,676989,1473233922,912605,1473233922,912605,NaN,2016-06-21 22:02:01,...,0.009930,0.002847,-0.024984,0.018332,0.019608,0.013177,0.051668,0.023849,0.030201,-0.086340


In [16]:
pipeline, vectorized_test = lsa(unpredicted, pipeline)
vectorized_test.tail()

,index,sid,id,position,created_at,created_meta,updated_at,updated_meta,meta,Date received,...,190,191,192,193,194,195,196,197,198,199
40872,623378,1011507,BB3F28BA-D61E-44E5-A37A-4C01A851FAA7,1011507,1490626002,912605,1490626002,912605,NaN,2017-03-19 03:31:19,...,0.034792,0.023186,0.013092,-0.078959,-0.019740,0.044456,-0.023159,0.052129,0.051821,-0.065521
40873,622857,1011079,9BE91B24-F1C4-4A63-BE54-27F020659DBC,1011079,1490626000,912605,1490626000,912605,NaN,2017-03-19 17:43:38,...,-0.031394,0.030891,-0.065365,-0.020553,-0.017464,0.080740,-0.002054,0.043343,0.004345,0.002092
40874,622979,1011181,034C182E-36EC-458E-AA0D-7A4398CF152F,1011181,1490626000,912605,1490626000,912605,NaN,2017-03-19 19:08:13,...,-0.066013,0.117919,0.011882,-0.206303,-0.015352,-0.048520,0.030296,0.127439,-0.027035,-0.001911
40875,627449,1013441,F3EBEAD9-049C-4B45-B4D6-B41D925D902C,1013441,1490626005,912605,1490626005,912605,NaN,2017-03-20 11:12:52,...,0.028614,0.021991,0.040306,-0.020897,0.055943,0.008045,0.049387,-0.024154,-0.028773,0.001851
40876,625730,1012954,72C8CBDA-F83B-45D7-BCB9-EECA231A4FD7,1012954,1490626005,912605,1490626005,912605,NaN,2017-03-20 15:31:42,...,-0.013180,0.006567,-0.048714,0.043359,0.020348,0.006093,-0.005703,0.021440,0.015062,-0.003576


In [17]:
nbrs = NearestNeighbors(n_neighbors=2, n_jobs=-1).fit(vectorized_train.iloc[:,-200:].values)

In [18]:
nearest = pd.DataFrame(nbrs.kneighbors(vectorized_test.iloc[:,-200:].values, return_distance=False))

In [19]:
def find_match(entry):
    return vectorized_train.iloc[entry]['Company response to consumer']
print(vectorized_test.shape)
print(nearest.shape)
vectorized_test['NN Prediction 1'] = nearest.iloc[:,0].map(lambda x: find_match(x))
vectorized_test['NN Prediction 2'] = nearest.iloc[:,1].map(lambda x: find_match(x))
vectorized_test['NN Prediction 1'].tail()

(40877, 231)
(40877, 2)


40872    Closed with monetary relief
40873        Closed with explanation
40874        Closed with explanation
40875        Closed with explanation
40876        Closed with explanation
Name: NN Prediction 1, dtype: object

In [20]:
pd.crosstab(vectorized_test['Company response to consumer'], vectorized_test['NN Prediction 1'])

NN Prediction 1,Closed with explanation,Closed with monetary relief,Closed with non-monetary relief
Company response to consumer,,,
Closed with explanation,23907,2708,4616
Closed with monetary relief,2172,1370,331
Closed with non-monetary relief,3761,303,1709


In [21]:
pd.crosstab(vectorized_test['Company response to consumer'], vectorized_test['NN Prediction 2'])

NN Prediction 2,Closed with explanation,Closed with monetary relief,Closed with non-monetary relief
Company response to consumer,,,
Closed with explanation,23897,2624,4710
Closed with monetary relief,2189,1369,315
Closed with non-monetary relief,3751,284,1738


In [82]:
grouped_train = new_training.groupby(['combined_product','combined_issue'])
grouped_model = {}
def tokenize(words, vocab):
    for word in words.split(' '):
        vocab.add(word)

for name, group, in grouped_train:
    vocab = set([])
    group['Consumer complaint narrative'].apply(lambda x: tokenize(x, vocab))
    words = len(vocab)
    dims = int(0.8 * words)
    if dims > 200:
        dims = 200
    print("%s: %s"%(dims, name))
    grouped_model[name] = (lsa(group, dimensions=dims, short=True), dims)

200: ('Bank account or service-(CD) Certificate of deposit', 'Account opening; closing; or management-nan')
200: ('Bank account or service-(CD) Certificate of deposit', 'Deposits and withdrawals-nan')
97: ('Bank account or service-(CD) Certificate of deposit', 'Making/receiving payments; sending money-nan')
197: ('Bank account or service-(CD) Certificate of deposit', 'Using a debit or ATM card-nan')


/Users/jponder/anaconda/lib/python3.5/site-packages/sklearn/decomposition/truncated_svd.py:186: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var


200: ('Bank account or service-Cashing a check without an account', 'Account opening; closing; or management-nan')
200: ('Bank account or service-Cashing a check without an account', 'Deposits and withdrawals-nan')
200: ('Bank account or service-Cashing a check without an account', 'Making/receiving payments; sending money-nan')
112: ('Bank account or service-Cashing a check without an account', 'Problems caused by my funds being low-nan')
164: ('Bank account or service-Cashing a check without an account', 'Using a debit or ATM card-nan')
200: ('Bank account or service-Checking account', 'Account opening; closing; or management-nan')
200: ('Bank account or service-Checking account', 'Deposits and withdrawals-nan')
200: ('Bank account or service-Checking account', 'Making/receiving payments; sending money-nan')
200: ('Bank account or service-Checking account', 'Problems caused by my funds being low-nan')
200: ('Bank account or service-Checking account', 'Using a debit or ATM card-nan')


/Users/jponder/anaconda/lib/python3.5/site-packages/sklearn/decomposition/truncated_svd.py:186: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/Users/jponder/anaconda/lib/python3.5/site-packages/sklearn/decomposition/truncated_svd.py:186: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/Users/jponder/anaconda/lib/python3.5/site-packages/sklearn/decomposition/truncated_svd.py:186: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var


84: ('Consumer Loan-Title loan', "Can't stop charges to bank account-nan")
36: ('Consumer Loan-Title loan', 'Charged bank acct wrong day or amt-nan')
200: ('Consumer Loan-Title loan', "Charged fees or interest I didn't expect-nan")
79: ('Consumer Loan-Title loan', 'Lender damaged or destroyed vehicle-nan')
200: ('Consumer Loan-Title loan', 'Lender repossessed or sold the vehicle-nan')
200: ('Consumer Loan-Title loan', 'Payment to acct not credited-nan')


...

200: ('Prepaid card-Payroll card', 'Managing; opening; or closing account-nan')
135: ('Prepaid card-Payroll card', 'Overdraft; savings or rewards features-nan')
200: ('Prepaid card-Payroll card', 'Unauthorized transactions/trans. issues-nan')


/Users/jponder/anaconda/lib/python3.5/site-packages/sklearn/decomposition/truncated_svd.py:186: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var


45: ('Prepaid card-Transit card', 'Adding money-nan')
200: ('Prepaid card-Transit card', 'Fraud or scam-nan')
51: ('Prepaid card-Transit card', 'Managing; opening; or closing account-nan')
200: ('Prepaid card-Transit card', 'Unauthorized transactions/trans. issues-nan')
200: ('Student loan-Federal student loan servicing', "Can't repay my loan-Can't decrease my monthly payments")


/Users/jponder/anaconda/lib/python3.5/site-packages/sklearn/decomposition/truncated_svd.py:186: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var


200: ('Student loan-Federal student loan servicing', "Can't repay my loan-Can't get flexible payment options")
200: ('Student loan-Federal student loan servicing', "Can't repay my loan-Can't temporarily postpone payments")
200: ('Student loan-Federal student loan servicing', "Dealing with my lender or servicer-Don't agree with fees charged")
200: ('Student loan-Federal student loan servicing', 'Dealing with my lender or servicer-Having problems with customer service')
200: ('Student loan-Federal student loan servicing', 'Dealing with my lender or servicer-Keep getting calls about my loan')
200: ('Student loan-Federal student loan servicing', 'Dealing with my lender or servicer-Need information about my balance/terms')
200: ('Student loan-Federal student loan servicing', 'Dealing with my lender or servicer-Received bad information about my loan')
200: ('Student loan-Federal student loan servicing', 'Dealing with my lender or servicer-Trouble with how payments are handled')
200: ('Studen

In [83]:
group_test = unpredicted.groupby(['combined_product','combined_issue'])
grouped_test = {}
bad_keys = 0
for name, group in group_test:
    try:
        grouped_test[name] = (lsa(group, grouped_model[name][0][0], short=True), grouped_model[name][1])
    except Exception as e:
        print(e)
        print(group.size)
        

('Bank account or service-(CD) Certificate of deposit', 'Problems caused by my funds being low-nan')
60
('Consumer Loan-Pawn loan', 'Lender damaged or destroyed property-nan')
30
('Consumer Loan-Personal line of credit', 'Problems when you are unable to pay-nan')
30
('Consumer Loan-Title loan', "Received a loan I didn't apply for-nan")
30
('Debt collection-Auto', 'False statements or representation-Indicated committed crime not paying')
30
('Debt collection-Auto', "Taking/threatening an illegal action-Sued where didn't live/sign for debt")
30
('Debt collection-Federal student loan', 'Disclosure verification of debt-Not disclosed as an attempt to collect')
60
('Debt collection-Federal student loan', 'False statements or representation-Impersonated an attorney or official')
30
('Debt collection-Medical', 'Improper contact or sharing of info-Contacted me instead of my attorney')
30
('Debt collection-Non-federal student loan', 'False statements or representation-Impersonated an attorney or

In [86]:
nbr_groups = {}
for key, value in grouped_model.items():
    dims = value[1]
#    print("%s: %s" % (dims, key))
#    print(value[0][1])
    nbr_groups[key] = NearestNeighbors(n_neighbors=1, n_jobs=1).fit(value[0][1].iloc[:,1:].values)

In [100]:
nbr_results = pd.DataFrame()
for key, value in grouped_test.items():
    results = pd.DataFrame(nbr_groups[key].kneighbors(value[0][1].iloc[:,1:].values, return_distance=False))
    predictions = results.iloc[:,0].map(lambda x: grouped_model[key][0][1].iloc[x,0])
    original = value[0][1].iloc[:,0]
    values = pd.concat([predictions, original], axis=1)
    confusion = pd.crosstab(values.iloc[:,0], values.iloc[:,1])
    print(confusion)
    nbr_results = pd.concat([nbr_results, values])

Company response to consumer  Closed with explanation
0                                                    
Closed with explanation                             1
Company response to consumer     Closed with explanation  \
0                                                          
Closed with explanation                                7   
Closed with non-monetary relief                        0   

Company response to consumer     Closed with non-monetary relief  
0                                                                 
Closed with explanation                                        0  
Closed with non-monetary relief                                1  
Company response to consumer     Closed with explanation  \
0                                                          
Closed with explanation                              238   
Closed with monetary relief                           85   
Closed with non-monetary relief                       69   

Company response to consumer

In [102]:
pd.crosstab(nbr_results.iloc[:,0], nbr_results.iloc[:,1])

Company response to consumer,Closed with explanation,Closed with monetary relief,Closed with non-monetary relief
0,,,
Closed with explanation,23670,2139,3683
Closed with monetary relief,2686,1395,272
Closed with non-monetary relief,4856,336,1816
